#### ETL Overview 

_Data_

There are two main data sources for my portion of the project:
* [USCRN Hourly Historical Weather Data](https://www.ncei.noaa.gov/pub/data/uscrn/products/hourly02/): This page contains hourly weather data from the U.S. Climate Reference Network / U.S. Regional Climate Reference Network (USCRN/USRCRN) stored as tabular .txt files. 
* [NWS Forecasts](https://forecast.weather.gov/MapClick.php?lat=60.7506&lon=-160.5006&unit=0&lg=english&FcstType=digital): The National Weather Service has forecast offices in Fairbanks and Anchorage which provide hourly forecasts for various locations in Alaska. 

_USCRN pipeline_

For data that's already available: 

0. Save column names and description from [headers.txt](https://www.ncei.noaa.gov/pub/data/uscrn/products/hourly02/headers.txt) to .csv
1.  Scrape all existing USCRN data from weather stations in Alaska and save locally to a .csv
       - For each year ([2000-2023](https://www.ncei.noaa.gov/pub/data/uscrn/products/hourly02/)): 
         - Find all filenames containing 'AK'
         - For each 'AK' file: 
           - Retrieve .txt data as dataframe 
           - Add column headers from `headers.txt`
           - Add a location column based on filename 
           - Append to local .csv file
2.  Upload .csv to Big Query 

For future data, write a dag with tasks: 
1. getUpdates()
    -  Checks the value for 'Last modified' on the current year's [index page](https://www.ncei.noaa.gov/pub/data/uscrn/products/hourly02/2023/) 
    -  If more recent than the last recorded value: 
       -  For every AK dataset on page: 
          -  Retrieve the .txt file 
          -  Filter for records after current 'Last modified' value
          -  Write new records to .csv  
       -  Store the new 'Last modified' value 
       -  Trigger next task
    -  Else, pass
2. upload() 
   - Upload (append) .csv of new records to BigQuery table 

_NWS Forecast 